In [1]:
import pandas as pd
import geopandas as gpd
import openrouteservice as ors
import folium
import sys
import concurrent.futures
from tqdm import tqdm
from threading import Semaphore
import json
import folium
import openrouteservice
import json
import time
import openrouteservice
import geopandas as gpd
import folium
from shapely.geometry import Point
from shapely.geometry import shape

from shapely import wkt
from openrouteservice.exceptions import ApiError
sys.path.append('../scripts')
from utilities import get_vic_subs_as_csv

In [2]:
client = ors.Client(key='5b3ce3597851110001cf6248c33b591496c2484889d49c9f98e8e699')

In [3]:
input_file = '../data/raw/current_rent_info.json'
output_file = '../data/raw/coordinates_by_address.json'
with open(input_file, 'r') as file:
    data = json.load(file)

In [4]:
# Define the rate limit
RATE_LIMIT = 100  # Requests per minute
REQUEST_INTERVAL = 60 / RATE_LIMIT  # Interval between requests in seconds

# Semaphore to control the number of concurrent requests
semaphore = Semaphore(3)  # Max number of concurrent threads
def get_coordinates(address):
    try:
        with semaphore:
            response = client.pelias_search(address)
            if response['features']:
                coords = response['features'][0]['geometry']['coordinates']
                return {'latitude': coords[1], 'longitude': coords[0]}
            else:
                return {'latitude': None, 'longitude': None}
    except Exception as e:
        print(f"Error fetching coordinates for address '{address}': {e}")
        return {'latitude': None, 'longitude': None}

def fetch_coordinates_for_item(url_details):
    url, details = url_details
    address = details.get('name')
    if address:
        coords = get_coordinates(address)
        details['coordinates'] = coords

def worker(url_details):
    fetch_coordinates_for_item(url_details)
    # Delay to prevent exceeding rate limits
    time.sleep(REQUEST_INTERVAL)

In [5]:
with open(input_file, 'r') as file:
    data = json.load(file)

In [ ]:
# Find coordinates based on address for houses
max_threads = 3

with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
    # Create a list of futures
    futures = [executor.submit(worker, (url, details)) for url, details in data.items()]

    # Wrap futures with tqdm to show progress
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing items"):
        try:
            future.result()
        except Exception as e:
            print(f"An error occurred: {e}")

# Save the updated data to a new JSON file
with open(output_file, 'w') as file:
    json.dump(data, file, indent=4)

print(f"Updated data with coordinates saved to {output_file}")

In [ ]:
with open('../data/raw/coordinates_by_address.json', 'r') as file:
    data = json.load(file)
coordinates = [
    [details['coordinates']['longitude'], details['coordinates']['latitude']]
    for url, details in list(data.items()) if details.get('coordinates')
]
# print(coordinates)
# print(len(coordinates))

In [ ]:
#Find closest train station
for coords in coordinates:
    pois = client.places(
        request='pois',  
        geojson={"type": "Point", "coordinates": coords},
        filter_category_ids=[604], 
        buffer=2000,
        limit=1, 
        sortby='distance'
    )
    
    print(f"POIs for coordinates {coords}:")
    
    if 'features' in pois:
        top_pois = pois['features'][:1]
        print(top_pois)
    else:
        print("No POIs found.")

In [ ]:
# VIC_COORDINATES = [-37.020100, 144.964600]
# m = folium.Map(location=VIC_COORDINATES, tiles='cartodbpositron', zoom_start=8)
# def fetch_isochrones(coordinate, retries=3, delay=10):
#     for attempt in range(retries):
#         try:
#             iso = client.isochrones(
#                 locations=[coordinate],
#                 profile='foot-walking',
#                 range=[900],
#                 validate=False
#             )
#             return iso['features']
#         except ApiError as e:
#             print(f"Error fetching isochrones for coordinate {coordinate}: {e}")
#             if attempt < retries - 1:
#                 time.sleep(delay)  # Wait before retrying
#                 delay *= 2  # Exponential backoff
#             else:
#                 print("Max retries reached. Skipping this coordinate.")
#                 return None
# def plot_isochrones(isochrones, map_obj):
#     if isochrones:
#         for isochrone in isochrones:
#             folium.Polygon(
#                 locations=[list(reversed(coord)) for coord in isochrone['geometry']['coordinates'][0]],
#                 fill='00ff00',
#                 opacity=0.5
#             ).add_to(map_obj)
            
# def plot_map(coordinates, map_obj, output_file, request_limit_per_minute):
#     request_interval = 60 / request_limit_per_minute  # Time between requests to adhere to rate limit

#     for idx, coord in enumerate(coordinates):
#         if idx % 10 == 0 and idx > 0:  # Print progress message every 10 coordinates
#             print(f"Processed {idx} coordinates.")
#         isochrones = fetch_isochrones(coord)
#         plot_isochrones(isochrones, map_obj)
#         time.sleep(request_interval)  # Sleep to respect rate limiting

#     # Save the map to an HTML file
#     map_obj.save(output_file)
#     print(f"Map with isochrones saved to {output_file}")     
    
# output_file = '../plots/output_map.html'
# request_limit_per_minute = 10
# plot_map(coordinates, m, output_file, request_limit_per_minute)       